In [1]:
from llama_cpp import Llama

llm = Llama(
    model_path="models/llama2/llama-2-13b-chat.ggmlv3.q4_K_M.bin"
)

llama.cpp: loading model from models/llama2/llama-2-13b-chat.ggmlv3.q4_K_M.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 15 (mostly Q4_K - Medium)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.09 MB
llama_model_load_internal: mem required  = 9336.95 MB (+ 1608.00 MB per state)
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 

In [2]:
content_to_sum = "This man is so fast. Once upon a time I saw him running around in flash time. One day, I found him dead."

In [3]:
for i in llm(
    prompt=f"### Instruction: Summarize the following, return the summarized text only: {content_to_sum}\n### Response:",
    stream=True,
):
    print(i["choices"][0]["text"])



This
 man
 was
 incred
ibly
 fast
 and
 could
 run
 at
 an
 aston
ishing
 speed
,
 almost
 like
 he
 was
 moving
 in
 flash
 time
.
 Unfortunately
,
 one
 day
 he
 was
 found
 dead
.




llama_print_timings:        load time = 26292.03 ms
llama_print_timings:      sample time =    14.96 ms /    35 runs   (    0.43 ms per token,  2339.10 tokens per second)
llama_print_timings: prompt eval time = 26291.91 ms /    50 tokens (  525.84 ms per token,     1.90 tokens per second)
llama_print_timings:        eval time = 21381.21 ms /    34 runs   (  628.86 ms per token,     1.59 tokens per second)
llama_print_timings:       total time = 47760.94 ms


In [5]:
from whispercpp import Whisper

w = Whisper.from_pretrained(
    "/home/vinhdq/matrix/ASR_and_Summarize/asr_summarize_app/models/whisper/ggml-tiny.en.bin"
)

whisper_init_from_file_no_state: loading model from '/home/vinhdq/matrix/ASR_and_Summarize/asr_summarize_app/models/whisper/ggml-tiny.en.bin'
whisper_model_load: loading model
whisper_model_load: n_vocab       = 51864
whisper_model_load: n_audio_ctx   = 1500
whisper_model_load: n_audio_state = 384
whisper_model_load: n_audio_head  = 6
whisper_model_load: n_audio_layer = 4
whisper_model_load: n_text_ctx    = 448
whisper_model_load: n_text_state  = 384
whisper_model_load: n_text_head   = 6
whisper_model_load: n_text_layer  = 4
whisper_model_load: n_mels        = 80
whisper_model_load: f16           = 1
whisper_model_load: type          = 1
whisper_model_load: mem required  =  129.00 MB (+    3.00 MB per decoder)
whisper_model_load: adding 1607 extra tokens
whisper_model_load: model ctx     =   73.58 MB
whisper_model_load: model size    =   73.54 MB
whisper_init_state: kv self size  =    2.62 MB
whisper_init_state: kv cross size =    8.79 MB


In [21]:
import ffmpeg
import numpy as np

try:
    y, _ = (
        ffmpeg.input(
            "/home/vinhdq/matrix/ASR_and_Summarize/whisper.cpp/samples/sample1.wav",
            threads=0,
        )
        .output("-", format="s16le", acodec="pcm_s16le", ac=1, ar=16000)
        .run(cmd=["ffmpeg", "-nostdin"], capture_stdout=True, capture_stderr=True)
    )
except ffmpeg.Error as e:
    raise RuntimeError(f"Failed to load audio: {e.stderr.decode()}") from e

arr = np.frombuffer(y, np.int16).flatten().astype(np.float32) / 32768.0

w.transcribe(arr)

" It wasn't like I was asking for the code to uniquely a bunker or anything like that, but the amount of resistance I got from this"

In [15]:
from io import StringIO
import sys

capture = StringIO()
sys.stdout = capture

w.stream_transcribe(5000, sample_rate=16000, step_ms=2500, n_threads=8)


init_device: Using device: '(null)' ...

Opened audio device: (id=2, name=(null))
  - sample_rate: 16000
  - format: 33056 (required: 33056)
  - channels: 1 (required: 1)
  - samples per frame: 1024


stream_transcribe: processing 40000 samples (step = 2.5 sec / len = 5.0 sec / keep = 0.2 sec), 8 threads, lang = en, task = transcribe, timestamps = 0 ...
stream_transcribe: n_new_line = 1, no_context = 1

=== Transcription starting now... ===



#### Experiment: get audio from microphone & transcribe


In [4]:
import pyaudio
import numpy as np


def process_audio(in_data, frame_count, time_info, status_flags):
    audio_data = np.frombuffer(in_data, dtype=np.float32)

    print(audio_data.shape)

    return (in_data, pyaudio.paContinue)


# Set up PyAudio
audio = pyaudio.PyAudio()

sr = 16000
sample_format = pyaudio.paFloat32
chunk = 1024
time_to_record = 3

stream = audio.open(
    format=sample_format,
    channels=1,
    rate=sr,
    input=True,
    output=False,
    frames_per_buffer=chunk,
)

ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card


In [20]:
sr / chunk * 0.25

3.90625

In [ ]:
data = []
while True:
    data = data[-int(sr * 0.25) // (chunk) :]
    for _ in range(sr // chunk * time_to_record):
        data.append(stream.read(chunk))
    print(w.transcribe(np.frombuffer(b"".join(data), dtype=np.float32)))

stream.stop_stream()
stream.close()
audio.terminate()

In [6]:
import wave

file = wave.open("sample.wav", "wb")
file.setnchannels(1)
file.setsampwidth(audio.get_sample_size(sample_format))
file.setframerate(sr)

# Write and Close the File
file.writeframes(b"".join(data))
file.close()

In [9]:
adu = np.frombuffer(b"".join(data), dtype=np.float32)

In [14]:
w.transcribe(adu)

" Sometimes I'm scared"